In [1]:
/**
 * @file portfolio-optimization-moead-cpp.ipynb
 *
 * A simple practical application of MultiObjective Decomposition Evolutionary Algorithm
 * - Differential Variant (MOEA/D-DE) in portfolio optimization. This example allows user to freely choose 
 * multiple stocks of their choice, which upon request, generates csv automagically 
 * via a helper function.
 *
 * The algorithm will try and optimize the trade-off between the returns and
 * volatility of the requested stocks.
 *
 * Data from Pandas Datareader library (https://pandas-datareader.readthedocs.io/en/latest/).
 */

In [2]:
#include <mlpack/xeus-cling.hpp>
#include <ensmallen.hpp>
#include "../utils/portfolio.hpp"
#include "../utils/front.hpp"
#include <ctime>

In [3]:
// Header files to create and show the plot.
#define WITHOUT_NUMPY 1
#include "matplotlibcpp.h"
#include "xwidgets/ximage.hpp"

namespace plt = matplotlibcpp;

In [4]:
using namespace ens;

In [5]:
using namespace ens::test;

Download backup data

In [6]:
!wget -q http://datasets.mlpack.org/portfolio.csv

### 1. Set the Model Parameters

In this section, we will select the parameters for the optimizer. Parameters include name of the stocks, starting date, ending date and Finance API Source.

In [7]:
//! Declare user specified data.
std::string stocks, startDate, endDate, dataSource;

In [8]:
stocks = "AAPL,NKE,GOOGL,AMZN";

//! Uncomment to set custom stocks.
// std::cout << "Type the name of symbol of the stocks via comma separated values (no spaces)" << std::endl;
// std::cin >> stocks;

We're setting the data source to Yahoo Finance API by default. For custom data-source, refer pandas-datareader documentation to get the exhaustive list of available API sources.

In [9]:
dataSource = "yahoo";

//! Uncomment to set custom data-source.
//std::cin >> dataSource;

In [10]:
startDate = "03/08/2018";

//! Uncomment to set custom start-date.
// std::cout << "Starting Date (YYYY/MM/DD or DD/MM/YYYY)" << std::endl;
// std::cin >> startDate;

Get the current date.

In [11]:
time_t current_time;
struct tm *timeinfo;
time(&current_time);

timeinfo = localtime(&current_time);

In [12]:
size_t day = timeinfo->tm_mday;
size_t month = timeinfo->tm_mon + 1;
size_t year = timeinfo->tm_year + 1900

In [13]:
std::stringstream dateToday;
dateToday << day << "/" << month << "/" << year;

endDate = dateToday.str();

//! Uncomment to set custom end-date.
// std::cout << "End Date (YYYY/MM/DD or DD/MM/YYYY)" << std::endl;
// std::cin >> endDate;

In [14]:
//! Uncomment to generate the csv file.
//! if(Portfolio(stocks, dataSource, startDate, endDate,"portfolio.csv"))
//!    std::cout << "Data query failed! Using backup csv." << std::endl;

### 2. Loading the Dataset

In this section, we will create a helper class which will generate the CSV file for us based on the parameters provided in previous section. This class would also define the objective functions in question, namely: Return and Volatility. Ideally, we would want to maximize the returns and reduce the volatility. Since our implementation of algorithm works on minimization of all objectives, we have appended negative sign to the returns objective which converts it into a minimization problem.

In [15]:
class PortfolioFunction
{
  public:
    PortfolioFunction(const std::string& stocks,
                      const std::string& dataSource,
                      const std::string& startDate,
                      const std::string& endDate)
    {
      returns.load("portfolio.csv", arma::csv_ascii);
      returns.shed_col(0);

      assets = returns.n_cols;
    }

    //! Get the starting point.
    arma::mat GetInitialPoint()
    {
      return arma::Col<double>(assets, 1, arma::fill::zeros);
    }
    
    struct VolatilityObjective
    {
        VolatilityObjective(arma::mat&& cov) : cov(cov) {}

        double Evaluate(const arma::mat& coords)
        {
          const double portfolioVolatility = arma::as_scalar(arma::sqrt(
                coords.t() * cov * 252 * coords));
          return portfolioVolatility;
        }

        arma::mat cov;
    };

    struct ReturnsObjective
    {
        ReturnsObjective(arma::mat&& mean) : mean(mean) {}

        double Evaluate(const arma::mat& coords)
        {
          const double portfolioReturns = arma::accu(mean % coords.t()) * 252;
          
          //! Negative sign appended to convert to minimization problem.
          return -portfolioReturns;
        }

        arma::mat mean;
    };


    //! Get objective functions.
    std::tuple<VolatilityObjective, ReturnsObjective> GetObjectives()
    {
      return std::make_tuple(VolatilityObjective(arma::cov(returns)), 
                             ReturnsObjective(arma::mean(returns)));
    }

    arma::mat returns;
    size_t assets;
};


//! The constructor will generate the csv file.
PortfolioFunction pf(stocks, dataSource, startDate, endDate);

const double lowerBound = 0;
const double upperBound = 1;

DefaultMOEAD moead(150, // Population size.
                   30,  // Max generations.
                   1.0,  // Crossover probability.
                   0.9, // Probability of sampling from neighbor.
                   20, // Neighborhood size.
                   20, // Perturbation index.
                   0.5, // Differential weight.
                   2, // Max childrens to replace parents.
                   1E-10, // epsilon.
                   lowerBound, // Lower bound.
                   upperBound // Upper bound.
                 );

NSGA2 nsga2(150, // population size: The number of candidates in the population.
            30, // max generations: The maximum number of generations allowed.
            0.5, // crossover probability: The probability that the elites reproduce.
            0.5, // mutation  probability: The probability of mutation among the elite.
            1e-3, // mutation strength: The strength of the mutation.
            1e-6, // epsilon: The minimum difference required to distinguish between two solutions.
            lowerBound, // lowerBound: Lower bound of the coordinates of the initial population
            upperBound // upperBound: Upper bound of the coordinates of the initial population
            );

arma::mat nsga2Coords = pf.GetInitialPoint();
arma::mat moeadCoords(nsga2Coords);
auto objectives = pf.GetObjectives();

### 3. Optimization 

There are plethora of algorithms to solve this family of problems often known as Multi Objective Problem (MOP). Multi Objective Evolutionary Algorithms (MOEA) are a set of algorithms which employs the concept of evolution to optimize these kind of problems. Notably, two algorithms are often used for this task:

a) NSGA-II: Non Dominated Sorting Algorithm - II.

b) MOEA/D-DE: Multi-Objective Evolutionary Algorithm via Decompostion - Differential Variant.

#### a) NSGA-II

NSGA-II is a classic go-to algorithm for MOPs. Each member of the population is assigned a fitness value and segragated into various fronts based on their fitness. This segragation mechanism is done using "Non Dominated Sorting" principle. It uses dominance relation to sort the population into various fronts and members ranked accordingly. The best Front is the one with the lowest rank.

#### b) MOEA/D - DE

MOEA/D-DE utilizes the concept of decomposition to tackle MOP. Unlike traditional algorithms like NSGA-II, it doens't rely on dominance relation. Instead, a set of "Reference Directions" are instantiated to frame it into a scalar optimization problem. The fitness value is assigned to the members in accordance to their performance in this framed optimization function. With the aid of Genetic Operators, offsprings replace the parent solutions if its fitness is superior.

MOEAD offers a plethora of Decomposition Functions and Reference Direction generators via templates. For our case, we've used the trusty ```DefaultMOEAD```. Read the class documentation for other options.

We would like to track the optimization process over the generations. For that let's create a container to store the current Pareto Front.

In [16]:
std::vector<arma::cube> nsga2Fronts{};
std::vector<arma::cube> moeadFronts{};

This data structure would then be passed on to the "QueryFront" Callback which will track the evolution for us.

Begin Optimization! (This will take a fair amount of time).

In [17]:
nsga2.Optimize(objectives, nsga2Coords, QueryFront(2, nsga2Fronts));

In [18]:
moead.Optimize(objectives, moeadCoords, QueryFront(2, moeadFronts));

Let's create an array to store the X and Y coordinates of all the Pareto Fronts.

In [19]:
std::stringstream nsga2FrontsX, nsga2FrontsY, moeadFrontsX, moeadFrontsY;

Convert to neccessary data structure.

In [20]:
void FillFront(std::stringstream& frontX,
               std::stringstream& frontY,
               std::vector<arma::cube>& frontList)
{
    size_t numFronts = frontList.size();
    
    for (size_t frontIdx = 0; frontIdx < numFronts; ++frontIdx)
    {
        size_t numPoints = frontList[frontIdx].n_slices;
        const arma::cube& front = frontList[frontIdx];
        for (size_t pointIdx = 0; pointIdx < numPoints; ++pointIdx)
        {
            if (pointIdx == numPoints - 1)
            {
                 frontX << front.slice(pointIdx)(0);
                 frontY << -front.slice(pointIdx)(1);
            }
            else
            {
                frontX << front.slice(pointIdx)(0) << ",";
                // Append negative again to restore the original 
                // maximization objective.
                frontY << -front.slice(pointIdx)(1) << ",";
            }
        }
    
        if (frontIdx == numFronts - 1) break;  
        
        frontX << ";";
        frontY << ";";   
    }
}

In [21]:
FillFront(nsga2FrontsX, nsga2FrontsY, nsga2Fronts);
FillFront(moeadFrontsX, moeadFrontsY, moeadFronts);

### 4.  Plotting

As said before, we desire higher returns and lower volatility. The Pareto Front generated gives an optimal set of solutions such that, higher volatility is traded-off with higher returns and vice-versa. Hence, all the solutions are "optimal". Based on user's preference, he/she can choose their solution from the generated front.

The Axis Labels are as follows:

X-Axis: Volatility

Y-Axis: Returns

We expect an increase in volatility with increase in returns.

In [22]:
//! A util to plot the evolution process gif.
Front(nsga2FrontsX.str(), nsga2FrontsY.str(), moeadFrontsX.str(), moeadFrontsY.str());

auto im = xw::image_from_file("fronts.gif").finalize();
im

A Jupyter widget with unique id: bbb8f72866d14b2b8dbacaaab4c86473

### 5. Final Thoughts

In this notebook, we've seen how a MultiObjective Optimization algorithm can help in investing in stocks. We specified our stocks and witnessed our algorithm optimize the returns vs volatility trade-off in action. From the evolution process depicted above, it can be deduced that:

a) The Pareto Front of MOEA/D-DE is uniformly distributed along the search space and continuous in nature. Whereas NSGA-II's Front is disconnected and the highly crowded in select areas.

b) The Pareto Front of MOEA/D-DE covers a larger expanse of the objective space compared to NSGA-II.

c) In terms of speed, MOEA/D-DE is much faster compared to NSGA-II.

Feel free to play around by selecting various stocks, start-date, end-date and see how the outcomes plays off. 